In [ ]:
# Importing PySpark into Google Collab

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
!tar xf spark-3.3.0-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
# Create spark sessionm
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load in data from Amazon TSV
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-08-31 00:00:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

In [5]:
# Create the vine_table DataFrame and filter based on total votes
vine_filtered_df = df.filter(df['total_votes'] > 20).select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

# Show the filtered DF
vine_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

In [9]:
# Filter DF to only show helpful_votes/ total_votes => 50
helpful_df = vine_filtered_df.filter((vine_filtered_df['helpful_votes'] / vine_filtered_df['total_votes']) >= .5)

# Show DF 
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

In [37]:
# Filtered by paid reviews
paid_reviews_df = helpful_df.filter(helpful_df['vine'] == 'Y')

# Show DF
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|
| ROB6VOW41E8O5|          4|          155|        172|   Y|                N|
|R1PSDADBJRKB7N|          5|          215|        234|   Y|                N|
|R379PSH3FBB6CG|          5|           22|         26|   Y|     

In [28]:
# Filter by non paid reviews
free_reviews_df = helpful_df.filter(helpful_df['vine'] == 'N')

# Show DF
free_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

In [56]:
# PAID REVIEWS STATS

# Total Reviews
paid_row_number = paid_reviews.count()

# 5 Star Reviews
five_star_paid = paid_reviews_df.filter(paid_reviews_df['star_rating'] == 5).count()

# 5 Star Percentage 
total_paid_reviews = paid_reviews_df.count()

paid_five_star_percentage = (five_star_paid / total_paid_reviews) * 100 

print('The total number of paid reviews is: ' + str(paid_row_number))
print('The total number of paid 5 star reviews is: ' + str(five_star_paid))
print('The percentage of paid 5 star reviews: ' + str(paid_five_star_percentage) + '%')

The total number of paid reviews is: 1032
The total number of 5 star reviews is: 440
The percentage of 5 star reviews: 42.63565891472868%


In [57]:
# FREE REVIEW STATS

# Total Reviews
free_row_number = free_reviews_df.count()

# 5 Star Reviews
five_star_free = free_reviews_df.filter(free_reviews_df['star_rating'] == 5).count()

# 5 Star Percentage 
total_free_reviews = free_reviews_df.count()

free_five_star_percentage = (five_star_free / total_free_reviews) * 100 

print('The total number of free reviews is: ' + str(free_row_number))
print('The total number of free 5 star reviews is: ' + str(five_star_free))
print('The percentage of free 5 star reviews: ' + str(free_five_star_percentage) + '%')

The total number of free reviews is: 46776
The total number of free 5 star reviews is: 21755
The percentage of free 5 star reviews: 46.50889344963229%
